In [20]:
from sklearn.linear_model import LogisticRegression 
from sklearn.datasets import load_iris 
from pprint import pprint
X,y=load_iris(return_X_y=True)
param_grid = {
    'penalty': 'l2',
    'C':.38,
    'solver':'liblinear',
    'max_iter': 100,
    'class_weight':'balanced'
}
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)
lr=LogisticRegression(**param_grid)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
report_dict=classification_report(y_test,y_pred,output_dict=True)
pprint(report_dict)

{'0': {'f1-score': 1.0, 'precision': 1.0, 'recall': 1.0, 'support': 10.0},
 '1': {'f1-score': 0.9411764705882353,
       'precision': 1.0,
       'recall': 0.8888888888888888,
       'support': 9.0},
 '2': {'f1-score': 0.9565217391304348,
       'precision': 0.9166666666666666,
       'recall': 1.0,
       'support': 11.0},
 'accuracy': 0.9666666666666667,
 'macro avg': {'f1-score': 0.9658994032395567,
               'precision': 0.9722222222222222,
               'recall': 0.9629629629629629,
               'support': 30.0},
 'weighted avg': {'f1-score': 0.9664109121909632,
                  'precision': 0.9694444444444444,
                  'recall': 0.9666666666666667,
                  'support': 30.0}}


In [21]:
import mlflow
mlflow.set_experiment("First Experiment")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

with mlflow.start_run(run_name='i am run name'):
    mlflow.log_params(param_grid)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recall_class_0': report_dict['0']['recall'],
        'recall_class_1': report_dict['1']['recall'],
        'recall_class_2': report_dict['2']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    mlflow.sklearn.log_model(lr, "Logistic Regression") 

2024/10/24 21:24:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/24 21:24:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run i am run name at: http://127.0.0.1:5000/#/experiments/131933486044636028/runs/be7b63a7dd954bcfbe6dce27d5a7c734.
2024/10/24 21:24:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/131933486044636028.


## multiple experiments  

In [39]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
models = [
    (
        "Logistic Regression", 
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test),
        {'C':.3,'solver':'liblinear'}
    ),
    (
        "Random Forest", 
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test),
        {'n_estimators':30, 'max_depth':1}
    ),
    (
        "XGBClassifier",
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test),
        {'use_label_encoder':False, 'eval_metric':'logloss'}
    )
]

In [36]:
reports = []
params_list=[]
for model_name, model, train_set, test_set,params in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    ### important 
    model.set_params(params)
    params_list.append(params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:02:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [38]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
# Initialize MLflow
mlflow.set_experiment("multi class")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    parameters=params_list[i]
    print(parameters)
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(parameters)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_2', report['2']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

{'C': 0.3, 'solver': 'liblinear'}


2024/10/24 22:03:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/24 22:03:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/601957073122898176/runs/9601b1f90fa44821be106b0be22837e9.
2024/10/24 22:03:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/601957073122898176.


{'n_estimators': 30, 'max_depth': 1}


2024/10/24 22:03:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/24 22:03:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: http://localhost:5000/#/experiments/601957073122898176/runs/f74b650603684a8f85f18763a04d30d8.
2024/10/24 22:03:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/601957073122898176.
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:03:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


{'use_label_encoder': False, 'eval_metric': 'logloss'}


2024/10/24 22:03:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/24 22:03:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/601957073122898176/runs/e1c98bd6783f41e4bb50405fc77f4dd0.
2024/10/24 22:03:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/601957073122898176.


## register model  (uploading an experiment to models)

In [45]:
# to put model that exists in experiments to models 
# when runnig more than one , it increase the version 
# model is created from experiment run 
model_name = 'Xgboost model'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/{model_name}'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID e1c98bd6783f41e4bb50405fc77f4dd0


Successfully registered model 'Xgboost model'.
2024/10/25 00:51:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Xgboost model, version 1
Created version '1' of model 'Xgboost model'.
2024/10/25 00:51:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/601957073122898176/runs/e1c98bd6783f41e4bb50405fc77f4dd0.
2024/10/25 00:51:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/601957073122898176.


## to load registeried model

In [51]:
# challenger 
# chambion 
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/601957073122898176/e1c98bd6783f41e4bb50405fc77f4dd0/artifacts/Xgboost model:
##### File  #####
API request to http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/601957073122898176/e1c98bd6783f41e4bb50405fc77f4dd0/artifacts/Xgboost model/ failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/601957073122898176/e1c98bd6783f41e4bb50405fc77f4dd0/artifacts/Xgboost%20model/ (Caused by ResponseError('too many 500 error responses'))

## transation model to production 

In [52]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'Xgboost model' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1729807525238, current_stage='None', description='', last_updated_timestamp=1729807525238, name='anomaly-detection-prod', run_id='e1c98bd6783f41e4bb50405fc77f4dd0', run_link='', source='models:/Xgboost model/1', status='READY', status_message='', tags={}, user_id='', version='1'>